## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [32]:
!pip install -U tensorflow --quiet

     |████████████████████████████████| 86.3MB 40kB/s 
     |████████████████████████████████| 3.8MB 29.4MB/s 
     |████████████████████████████████| 450kB 49.2MB/s 
     |████████████████████████████████| 81kB 8.6MB/s 
ERROR: tensorboard 2.0.1 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.6.3 which is incompatible.


In [0]:
import tensorflow as tf
tf.enable_eager_execution()

In [0]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default


### Collect Data

In [50]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [0]:
import pandas as pd

In [0]:
data = pd.read_csv('/gdrive/My Drive/AIML (1)/R6/prices.csv')

### Check all columns in the dataset

In [55]:
data.columns

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')

### Drop columns `date` and  `symbol`

In [0]:
data = data.drop(columns=['date','symbol'], axis=1)

In [57]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [58]:
X = data.iloc[0:1000, 0:4]
X.shape

(1000, 4)

In [59]:
Y = data.iloc[0:1000, 4]
Y.shape

(1000,)

In [0]:
Y = Y/1000000

In [61]:
Y.head()

0    2.1636
1    2.3864
2    2.4895
3    2.0063
4    1.4086
Name: volume, dtype: float64

### Divide the data into train and test sets

In [0]:
from sklearn.model_selection import train_test_split
import numpy as np

#### Convert Training and Test Data to numpy float32 arrays


In [0]:
X = np.array(X)
Y = np.array(Y)

In [0]:
X_train, X_test, Y_train,  Y_test = train_test_split(X, Y, test_size=0.3)

In [0]:
X_train = X_train.astype('float32')
Y_train = Y_train.astype('float32')
X_test = X_test.astype('float32')
Y_test = Y_test.astype('float32')

In [66]:
X_train

array([[ 26.42,  26.5 ,  26.42,  26.7 ],
       [ 31.78,  30.87,  30.25,  31.78],
       [ 85.42,  89.25,  85.  ,  89.38],
       ...,
       [105.63, 107.13, 104.11, 109.26],
       [ 65.  ,  65.89,  64.96,  66.  ],
       [113.38, 114.05, 109.64, 114.64]], dtype=float32)

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [0]:
from sklearn.preprocessing import Normalizer

In [68]:
X_train_std = Normalizer().transform(X_train)
X_train_std

array([[0.49829787, 0.4998067 , 0.49829787, 0.50357884],
       [0.50967485, 0.49508062, 0.4851373 , 0.50967485],
       [0.48930666, 0.51124585, 0.4869008 , 0.5119905 ],
       ...,
       [0.49568516, 0.5027242 , 0.48855233, 0.5127195 ],
       [0.49645388, 0.5032515 , 0.49614838, 0.5040916 ],
       [0.5019287 , 0.5048948 , 0.48537186, 0.50750667]], dtype=float32)

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [0]:
w = tf.zeros(shape=(4,1))
b = tf.zeros(shape=(1))

2.Define a function to calculate prediction

In [0]:
def prediction(x, w, b):
    
    xw_matmul = tf.matmul(x, w)
    y = tf.add(xw_matmul, b)
    
    return y

3.Loss (Cost) Function [Mean square error]

In [0]:
def loss(y_actual, y_predicted):
    
    diff = y_actual - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    
    return avg

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
def train(x, y_actual, w, b, learning_rate=0.01):
    
    #Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        
        t.watch([w,b])
        
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_actual, current_prediction)
    
    #Calculate Gradients for Loss with respect to Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    #Update Weights and Bias
    w = w - learning_rate*dw
    b = b - learning_rate*db
    
    return w, b

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [79]:
for i in range(100):
    w, b = train(X_train_std, Y_train, w, b, learning_rate=0.01)
    print('Current Loss on iteration', i, loss(Y_train, prediction(X_train_std, w, b)).numpy())

Current Loss on iteration 0 217.39024
Current Loss on iteration 1 217.39024
Current Loss on iteration 2 217.3902
Current Loss on iteration 3 217.39024
Current Loss on iteration 4 217.39027
Current Loss on iteration 5 217.39027
Current Loss on iteration 6 217.39027
Current Loss on iteration 7 217.39027
Current Loss on iteration 8 217.3903
Current Loss on iteration 9 217.39027
Current Loss on iteration 10 217.39027
Current Loss on iteration 11 217.3903
Current Loss on iteration 12 217.3903
Current Loss on iteration 13 217.39034
Current Loss on iteration 14 217.39034
Current Loss on iteration 15 217.39034
Current Loss on iteration 16 217.39034
Current Loss on iteration 17 217.39035
Current Loss on iteration 18 217.39037
Current Loss on iteration 19 217.39034
Current Loss on iteration 20 217.39037
Current Loss on iteration 21 217.39037
Current Loss on iteration 22 217.39038
Current Loss on iteration 23 217.3904
Current Loss on iteration 24 217.3904
Current Loss on iteration 25 217.3904
Cur

### Get the shapes and values of W and b

In [80]:
w.numpy()

array([[1.3375262],
       [1.3421087],
       [1.3240651],
       [1.3534039]], dtype=float32)

In [81]:
w.shape

TensorShape([Dimension(4), Dimension(1)])

In [82]:
b.numpy()

array([2.6786962], dtype=float32)

In [83]:
b.shape

TensorShape([Dimension(1)])

### Model Prediction on 1st Examples in Test Dataset

In [86]:
X_test_std = Normalizer().transform(X_test)
X_test_std

array([[0.5057782 , 0.49689114, 0.49140206, 0.5057782 ],
       [0.49297348, 0.5073806 , 0.49172068, 0.50769377],
       [0.49875224, 0.50414675, 0.49286726, 0.50414675],
       ...,
       [0.49641898, 0.50467783, 0.49359888, 0.5052016 ],
       [0.49435553, 0.503537  , 0.49435553, 0.50761765],
       [0.5006318 , 0.50109345, 0.49705422, 0.50120884]], dtype=float32)

In [103]:
prediction(X_test_std,w,b)[0]

<tf.Tensor: id=14136, shape=(1,), dtype=float32, numpy=array([5.35724], dtype=float32)>

## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [137]:
iris = pd.read_csv('/gdrive/My Drive/AIML (1)/R6/Iris.csv')
iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [0]:
iris=pd.get_dummies(iris, columns=['Species'])

In [139]:
iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species_Iris-setosa,Species_Iris-versicolor,Species_Iris-virginica
0,1,5.1,3.5,1.4,0.2,1,0,0
1,2,4.9,3.0,1.4,0.2,1,0,0
2,3,4.7,3.2,1.3,0.2,1,0,0
3,4,4.6,3.1,1.5,0.2,1,0,0
4,5,5.0,3.6,1.4,0.2,1,0,0


### Splitting the data into feature set and target set

In [0]:
X = iris.iloc[:,1:5]
Y = iris.iloc[:,5:]

In [141]:
X.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [142]:
Y.head()

,Species_Iris-setosa,Species_Iris-versicolor,Species_Iris-virginica
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0


In [0]:
X_train, X_test, Y_train,  Y_test = train_test_split(X, Y, test_size=0.3)

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [0]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Input(shape=(4,)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(3, activation='softmax'))

In [0]:
model.compile(optimizer='sgd', loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [125]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 4)                 16        
_________________________________________________________________
dense (Dense)                (None, 3)                 15        
Total params: 31
Trainable params: 23
Non-trainable params: 8
_________________________________________________________________


### Model Training 

In [146]:
model.fit(X_train,Y_train,          
          validation_data=(X_test, Y_test),
          epochs=200,
          batch_size=10)

Train on 105 samples, validate on 45 samples
Epoch 1/200
105/105 [==============================] - 0s 1ms/sample - loss: 0.2747 - acc: 0.8476 - val_loss: 0.2218 - val_acc: 0.9333
Epoch 2/200
105/105 [==============================] - 0s 231us/sample - loss: 0.3065 - acc: 0.8952 - val_loss: 0.2203 - val_acc: 0.9333
Epoch 3/200
105/105 [==============================] - 0s 225us/sample - loss: 0.2160 - acc: 0.9429 - val_loss: 0.2200 - val_acc: 0.9333
Epoch 4/200
105/105 [==============================] - 0s 237us/sample - loss: 0.3183 - acc: 0.8571 - val_loss: 0.2188 - val_acc: 0.9333
Epoch 5/200
105/105 [==============================] - 0s 231us/sample - loss: 0.2411 - acc: 0.9048 - val_loss: 0.2186 - val_acc: 0.9333
Epoch 6/200
105/105 [==============================] - 0s 229us/sample - loss: 0.2080 - acc: 0.9143 - val_loss: 0.2180 - val_acc: 0.9333
Epoch 7/200
105/105 [==============================] - 0s 226us/sample - loss: 0.2330 - acc: 0.8857 - val_loss: 0.2171 - val_acc: 0.933

### Model Prediction

In [147]:
prediction = model.predict(X_test)
prediction

array([[2.71421362e-04, 3.99893820e-01, 5.99834740e-01],
       [2.80434688e-05, 1.53511092e-01, 8.46460819e-01],
       [1.29338040e-03, 8.82631958e-01, 1.16074786e-01],
       [9.96696711e-01, 3.30297532e-03, 3.28596997e-07],
       [1.10734738e-02, 6.56010330e-01, 3.32916200e-01],
       [3.90869146e-03, 9.56084192e-01, 4.00071032e-02],
       [1.76896788e-02, 8.08060288e-01, 1.74249962e-01],
       [9.99059856e-01, 9.39564139e-04, 5.70690759e-07],
       [9.94191706e-01, 5.80772758e-03, 5.45625255e-07],
       [5.09410910e-03, 8.54830921e-01, 1.40075028e-01],
       [8.12880971e-05, 2.72049308e-01, 7.27869391e-01],
       [1.98562872e-02, 7.18199372e-01, 2.61944324e-01],
       [7.48563744e-03, 6.64624870e-01, 3.27889502e-01],
       [5.24321618e-03, 9.72777069e-01, 2.19797194e-02],
       [5.34347200e-05, 8.90639052e-03, 9.91040170e-01],
       [1.55859543e-02, 9.62953329e-01, 2.14607418e-02],
       [3.65914591e-03, 8.62632513e-01, 1.33708358e-01],
       [2.73421360e-03, 1.56491

### Save the Model

In [0]:
model.save('R6_iris_model.h5')

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?